<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=cd9350480697e8c3ff7bb7e8fa19bcdae17232faf6abf79ac4480fc11b54843e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-12 10:23:13
-------------------
qualified stocks: 84
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  5.59 L
Current:  1.44 C
-------------------
Today PnL: 85.87 K (0.6%)
Current PnL: -21.38 L (-14.01%)
CY Booked + Current PnL: -7.89 L (-5.17%)
-------------------
Total profit:  1.68 L
Total loss:  -23.06 L
-------------------
Total Booked + Current PnL: 19.09 L (15.19%)
Total Booked PnL: 40.47 L (32.22%)
Curr Year Booked PnL: 13.49 L (9.37%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.30 C
Est FTT PnL: 86.04 L (59.74%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.96%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,101.49,51.0,M-SC,2.55,86846.0,-13931.0,14026.0,0.20,-13.82,16.15,0.09,245.0,-0.99,0.63,14.10,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.65,45.0,X-MC,3.29,88335.0,-2343.0,17252.0,-0.36,-2.58,19.53,16.44,101.0,-0.14,0.64,17.07,X40,ATH,PHARMA
33,ICICIGI,2252.93,-13.14,57.0,X-MC,7.55,187581.0,13261.0,21947.0,1.05,7.61,11.70,20.19,91.0,0.60,1.35,23.87,X40,ATH,INSURANCE
41,ITC,452.00,-38.34,45.0,X-LC,0.83,198607.0,-1531.0,22423.0,-0.17,-0.76,11.29,10.44,4.0,-0.07,1.43,4.76,X40,NTT,FMCG
37,INDIGOPNTS,1408.00,127.68,78.0,M-SC,5.23,149116.0,-25443.0,25469.0,4.09,-14.58,17.08,0.02,221.0,-1.00,1.08,29.71,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIGOPNTS,1408.00,127.68,78.0,M-SC,5.23,149116.0,-25443.0,25469.0,4.09,-14.58,17.08,0.02,221.0,-1.00,1.08,29.71,OX40N,NTT,PAINTS
59,ROUTE,2227.21,-57.12,36.0,H-SC,23.24,70372.0,-62694.0,154579.0,3.68,-47.12,219.66,69.05,118.0,-0.41,0.51,3.68,SR,ATH,IT
16,CAMS,4762.00,-2.28,58.0,X-SC,1.23,261782.0,9926.0,52513.0,3.33,3.94,20.06,24.79,122.0,0.19,1.89,27.99,X40N,NTT,MISC
74,TITAGARH,1548.00,-5.17,52.0,H-SC,4.19,202692.0,-39485.0,150256.0,2.88,-16.30,74.13,45.74,158.0,-0.26,1.46,31.83,XY24,NTT,ENGINEERING
58,REPCOHOME,880.00,-52.31,66.0,H-SC,9.13,272445.0,-14060.0,271382.0,2.70,-4.91,99.61,89.82,134.0,-0.05,1.97,44.14,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VOLTAS,1530.00,-6.81,36.0,X-MC,4.02,195570.0,3828.0,33931.0,-2.75,2.00,17.35,19.69,99.0,0.11,1.41,8.93,XY25,NTT,AC
80,VBL,671.64,-18.08,48.0,X-LC,5.68,294820.0,-21022.0,133701.0,-1.81,-6.66,45.35,35.67,5.0,-0.16,2.13,6.28,X40N,ATH,FMCG
82,WHIRLPOOL,2270.00,-45.59,36.0,M-SC,8.83,112930.0,-1868.0,95911.0,-0.84,-1.63,84.93,81.92,223.0,-0.02,0.82,33.69,XR,NTT,DURABLES
72,TATAMOTORS,1065.00,-53.17,24.0,X-LC,37.92,147959.0,-128564.0,238643.0,-0.69,-46.49,161.29,39.81,54.0,-0.54,1.07,4.29,XY24,NTT,AUTO
17,CERA,9475.00,-26.97,33.0,H-SC,2.01,132952.0,-42951.0,84970.0,-0.66,-24.42,63.91,23.89,149.0,-0.51,0.96,14.82,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,WIPRO,420.0,-13.79,53.0,M-LC,5.75,151317.0,372.0,109084.0,0.98,0.25,72.09,72.51,53.0,0.0,1.09,6.28,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,101.49,51.0,M-SC,2.55,86846.0,-13931.0,14026.0,0.20,-13.82,16.15,0.09,245.0,-0.99,0.63,14.10,OX40N,NTT,DURABLES
37,INDIGOPNTS,1408.0,127.68,78.0,M-SC,5.23,149116.0,-25443.0,25469.0,4.09,-14.58,17.08,0.02,221.0,-1.00,1.08,29.71,OX40N,NTT,PAINTS
45,KANSAINER,340.0,-68.34,39.0,H-SC,2.28,218817.0,-50850.0,87177.0,-0.30,-18.86,39.84,13.47,138.0,-0.58,1.58,11.26,XY24,NTT,PAINTS
71,TATAELXSI,9161.0,-24.21,50.0,H-MC,6.36,102610.0,-29225.0,71447.0,2.21,-22.17,69.63,32.03,98.0,-0.41,0.74,14.93,OX40N,NTT,IT
65,SIS,528.0,2029.02,48.0,H-SC,2.20,85910.0,-25122.0,48728.0,0.87,-22.63,56.72,21.26,156.0,-0.52,0.62,16.07,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,WIPRO,420.00,-13.79,53.0,M-LC,5.75,151317.0,372.0,109084.0,0.98,0.25,72.09,72.51,53.0,0.00,1.09,6.28,XR,NTT,IT
36,INDIAMART,4810.62,-52.77,59.0,H-SC,4.89,131694.0,8358.0,123266.0,1.34,6.78,93.60,106.72,119.0,0.07,0.95,31.74,AR,ATH,MISC
60,SAMMAANCAP,326.00,-193.18,57.0,M-SC,35.70,161640.0,12420.0,131753.0,1.84,8.32,81.51,96.62,208.0,0.09,1.17,74.93,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-52.77,59.0,H-SC,4.89,131694.0,8358.0,123266.0,1.34,6.78,93.60,106.72,119.0,0.07,0.95,31.74,AR,ATH,MISC
83,WIPRO,420.00,-13.79,53.0,M-LC,5.75,151317.0,372.0,109084.0,0.98,0.25,72.09,72.51,53.0,0.00,1.09,6.28,XR,NTT,IT
82,WHIRLPOOL,2270.00,-45.59,36.0,M-SC,8.83,112930.0,-1868.0,95911.0,-0.84,-1.63,84.93,81.92,223.0,-0.02,0.82,33.69,XR,NTT,DURABLES
46,KPIGREEN,731.05,3.26,45.0,H-SC,10.19,123077.0,-2220.0,61145.0,0.43,-1.77,49.68,47.03,141.0,-0.04,0.89,54.10,MH,ATH,POWER
35,IEX,219.00,-33.63,47.0,H-SC,15.86,189437.0,-10517.0,107752.0,0.19,-5.26,56.88,48.63,115.0,-0.10,1.37,5.69,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.17,24.0,X-LC,37.92,147959.0,-128564.0,238643.0,-0.69,-46.49,161.29,39.81,54.0,-0.54,1.07,4.29,XY24,NTT,AUTO
25,GILLETTE,11206.78,-17.81,25.0,X-MC,5.74,252765.0,-1131.0,83438.0,1.70,-0.45,33.01,32.42,105.0,-0.01,1.83,14.59,X40,ATH,FMCG
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86860.0,-41810.0,87112.0,-0.04,-32.49,100.29,35.20,219.0,-0.48,0.63,0.00,X40,NTT,FOOTWEAR
1,ABB,7934.00,-42.78,32.0,H-LC,4.98,238968.0,-22651.0,141875.0,-0.27,-8.66,59.37,45.57,7.0,-0.16,1.73,1.68,AR,NTT,ELECTRICAL
44,JSWINFRA,342.00,-24.16,32.0,X-MC,4.71,190451.0,-9896.0,40395.0,-0.44,-4.94,21.21,15.23,178.0,-0.24,1.38,25.72,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-38.34,45.0,X-LC,0.83,198607.0,-1531.0,22423.0,-0.17,-0.76,11.29,10.44,4.0,-0.07,1.43,4.76,X40,NTT,FMCG
52,PGHH,17907.65,-32.00,38.0,X-MC,1.10,196800.0,-4020.0,71812.0,1.56,-2.00,36.49,33.76,80.0,-0.06,1.42,3.09,X40,ATH,FMCG
64,SIEMENS,4671.50,-5.61,39.0,H-LC,1.67,151750.0,-34345.0,81824.0,0.43,-18.46,53.92,25.51,15.0,-0.42,1.10,11.83,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-8.73,38.0,X-MC,3.31,304395.0,-23596.0,130129.0,0.41,-7.19,42.75,32.48,92.0,-0.18,2.20,3.41,X40,ATH,ELECTRICAL
77,UNITDSPR,1644.00,-7.24,59.0,X-LC,3.47,243122.0,8978.0,39653.0,-0.07,3.83,16.31,20.77,86.0,0.23,1.76,11.02,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86860.0,-41810.0,87112.0,-0.04,-32.49,100.29,35.20,219.0,-0.48,0.63,0.00,X40,NTT,FOOTWEAR
53,QUESS,986.00,-51.52,32.0,X-SC,37.14,47935.0,-17071.0,168003.0,0.18,-26.26,350.48,232.18,198.0,-0.10,0.35,0.18,XY24,NTT,MISC
1,ABB,7934.00,-42.78,32.0,H-LC,4.98,238968.0,-22651.0,141875.0,-0.27,-8.66,59.37,45.57,7.0,-0.16,1.73,1.68,AR,NTT,ELECTRICAL
10,BAJAJHFL,181.50,-15.96,38.0,X-MC,6.33,176956.0,-23550.0,120702.0,0.20,-11.75,68.21,48.45,90.0,-0.20,1.28,2.04,X40N,ATH,FINANCE
20,COLPAL,3726.84,-6.08,45.0,X-MC,7.00,219020.0,-44345.0,153664.0,0.37,-16.84,70.16,41.51,84.0,-0.29,1.58,2.78,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-51.52,32.0,X-SC,37.14,47935.0,-17071.0,168003.0,0.18,-26.26,350.48,232.18,198.0,-0.10,0.35,0.18,XY24,NTT,MISC
57,RELAXO,1176.00,-39.97,46.0,X-SC,3.69,81799.0,-63361.0,139287.0,0.74,-43.65,170.28,52.31,136.0,-0.45,0.59,9.49,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86860.0,-41810.0,87112.0,-0.04,-32.49,100.29,35.20,219.0,-0.48,0.63,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.65,45.0,X-MC,3.29,88335.0,-2343.0,17252.0,-0.36,-2.58,19.53,16.44,101.0,-0.14,0.64,17.07,X40,ATH,PHARMA
32,HONAUT,58357.33,-23.99,40.0,X-SC,0.92,107160.0,-20778.0,67907.0,0.61,-16.24,63.37,36.84,143.0,-0.31,0.77,9.71,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-28.04,54.0,X-LC,9.35,286418.0,-59538.0,126253.0,0.72,-17.21,44.08,19.28,1.0,-0.47,2.07,5.88,X40,ATH,IT
39,INFY,2275.00,-15.29,67.0,X-LC,4.54,329681.0,16793.0,154884.0,1.14,5.37,46.98,54.87,3.0,0.11,2.38,14.09,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,X-LC,0.83,198607.0,-1531.0,22423.0,-0.17,-0.76,11.29,10.44,4.0,-0.07,1.43,4.76,X40,NTT,FMCG
80,VBL,671.64,-18.08,48.0,X-LC,5.68,294820.0,-21022.0,133701.0,-1.81,-6.66,45.35,35.67,5.0,-0.16,2.13,6.28,X40N,ATH,FMCG
1,ABB,7934.00,-42.78,32.0,H-LC,4.98,238968.0,-22651.0,141875.0,-0.27,-8.66,59.37,45.57,7.0,-0.16,1.73,1.68,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,26.24,76781.0,-36768.0,76819.0,0.23,-32.38,100.05,35.27,268.0,-0.48,0.55,87.92,XR,NTT,HOTELS
7,ASIANTILES,137.00,7600.00,69.0,L-SC,15.02,84468.0,-9342.0,85963.0,0.59,-9.96,101.77,81.67,269.0,-0.11,0.61,63.58,XR,NTT,CERAMICS
49,MASFIN,398.61,-20.05,46.0,H-SC,7.85,90750.0,-7230.0,28831.0,0.02,-7.38,31.77,22.05,152.0,-0.25,0.66,31.91,XR,ATH,FINANCE
46,KPIGREEN,731.05,3.26,45.0,H-SC,10.19,123077.0,-2220.0,61145.0,0.43,-1.77,49.68,47.03,141.0,-0.04,0.89,54.10,MH,ATH,POWER
36,INDIAMART,4810.62,-52.77,59.0,H-SC,4.89,131694.0,8358.0,123266.0,1.34,6.78,93.60,106.72,119.0,0.07,0.95,31.74,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,VAIBHAVGBL,521.00,69.70,59.0,H-SC,6.56,147148.0,-35627.0,146692.0,1.30,-19.49,99.69,60.77,125.0,-0.24,1.06,32.52,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7600.00,69.0,L-SC,15.02,84468.0,-9342.0,85963.0,0.59,-9.96,101.77,81.67,269.0,-0.11,0.61,63.58,XR,NTT,CERAMICS
37,INDIGOPNTS,1408.00,127.68,78.0,M-SC,5.23,149116.0,-25443.0,25469.0,4.09,-14.58,17.08,0.02,221.0,-1.00,1.08,29.71,OX40N,NTT,PAINTS
5,ANGELONE,3033.00,23.98,71.0,X-SC,6.89,219826.0,28820.0,28885.0,1.49,15.09,13.14,30.21,157.0,1.00,1.59,39.12,X40N,NTT,FINANCE
6,ASIANPAINT,3465.66,-7.31,75.0,X-LC,6.81,233939.0,-17829.0,71047.0,0.05,-7.08,30.37,21.13,27.0,-0.25,1.69,25.83,X40,ATH,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.51
1,25,45.97
2,50,77.57


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.11
LC    29.88
MC    25.02
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.04
X40      19.94
X40N     12.86
XY25     10.53
XR       10.02
AR        7.97
OX40N     6.81
X200      1.89
SR        1.06
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.76
X-MC    22.25
X-LC    21.59
M-SC    13.03
X-SC     5.82
H-LC     4.72
M-LC     2.54
H-MC     2.42
L-SC     1.50
L-LC     1.03
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.45,-4.20,37.90
IT,13.14,-15.55,77.40
FINANCE,12.61,-8.37,57.49
MISC,7.98,-17.62,76.94
PAINTS,5.96,-11.92,28.60
ELECTRICAL,5.80,-12.64,52.57
INSURANCE,4.62,1.25,33.55
AUTO,2.90,-43.08,102.79
BANKS,2.82,-37.06,126.38


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3112329.0,20
XR,1269437.0,13
AR,1186061.0,9
X40,930272.0,13
X40N,689607.0,9
OX40N,541582.0,9
XY25,478110.0,7
SR,273279.0,2
X200,62513.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3402965.0,24
X-MC,1327502.0,15
M-SC,1242929.0,15
X-LC,1051216.0,12
X-SC,543707.0,6
H-LC,286212.0,3
L-SC,252376.0,3
H-MC,252089.0,2
M-LC,153794.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1172303.0      6
           AR         838529.0      5
           XR         799817.0      7
M-SC       XY24       721311.0      6
X-MC       XY24       563611.0      4
X-LC       X40        427961.0      5
X-MC       X40        415199.0      7
X-LC       XY24       306459.0      2
X-SC       X40N       288592.0      4
H-SC       SR         273279.0      2
           OX40N      257892.0      3
X-LC       X40N       239918.0      3
H-LC       AR         223699.0      2
X-MC       XY25       187595.0      2
H-MC       XY24       180642.0      1
X-SC       XY24       168003.0      1
L-SC       XR         162782.0      2
X-MC       X40N       161097.0      2
M-SC       XR         143383.0      2
           XY25       131753.0      1
           AR         123833.0      2
           OX40N      122649.0      4
M-LC       XR         109084.0      1
L-SC       OX40N       89594.0      1
X-SC       X40         87112.0      1
X-LC       XY25        76878.0      2
H-MC       OX40N       71447.0      1
H-LC       X200        62513.0      1
H-SC       MH          61145.0      1
L-MC       XR          54371.0      1
M-LC       XY25        44710.0      1
L-LC       XY25        37174.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
